In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Define a simple black-box model
class BlackBoxModel(nn.Module):
    def __init__(self):
        super(BlackBoxModel, self).__init__()
        self.fc = nn.Linear(10, 1)  # Assume input dimension is 10 for this example

    def forward(self, x):
        return self.fc(x)

def calculate_objective(X, X_prime, mu, nu, model, alpha, lambda_, epsilon):
    # Calculate the first term of the objective
    term1 = torch.sum(alpha @ (X - X_prime).T ** 2 * mu)

    # Calculate the second term of the objective
    term2 = lambda_ * (epsilon - torch.sum((model(X) - model(X_prime)) ** 2 * nu))

    # The total objective is the sum of the two terms
    objective = term1 + term2

    return objective

def optimize(X, X_prime, mu, nu, model, alpha, lambda_, epsilon, lr=0.01, max_iterations=1000):
    optimizer = optim.Adam([X], lr=lr)

    for iteration in range(max_iterations):
        optimizer.zero_grad()

        objective = calculate_objective(X, X_prime, mu, nu, model, alpha, lambda_, epsilon)
        objective.backward()
        optimizer.step()

        if iteration % 100 == 0:  # Print the objective every 100 iterations
            print(f"Iteration {iteration}, Objective {objective.item()}")

    return X

# Generate synthetic data
np.random.seed(0)
X = np.random.randn(100, 10)
X_prime = np.random.randn(100, 10)
mu = np.random.rand(100)
nu = np.random.rand(100)
alpha = np.random.randn(10)
epsilon = 1.0
lambda_ = 1.0

# Convert numpy arrays to torch tensors
X = torch.from_numpy(X).float().requires_grad_(True)
X_prime = torch.from_numpy(X_prime).float()
mu = torch.from_numpy(mu).float()
nu = torch.from_numpy(nu).float()
alpha = torch.from_numpy(alpha).float()

# Initialize the black-box model
model = BlackBoxModel()

# Run the optimizer
X_opt = optimize(X, X_prime, mu, nu, model, alpha, lambda_, epsilon)

print("Optimized X:")
print(X_opt)


Iteration 0, Objective -2302.5458984375
Iteration 100, Objective -23035.8046875
Iteration 200, Objective -85641.6875
Iteration 300, Objective -194893.546875
Iteration 400, Objective -350050.9375
Iteration 500, Objective -549557.375
Iteration 600, Objective -791735.625
Iteration 700, Objective -1074975.0
Iteration 800, Objective -1397763.875
Iteration 900, Objective -1758690.75
Optimized X:
tensor([[ 17.3844,  15.9071,  16.6451, -12.9078,  17.1008, -16.6038,  16.5064,
         -15.7756,  15.5132, -15.1987],
        [-15.6625, -14.4250, -15.0417,  15.2437, -14.9267,  16.2349, -14.0650,
          15.6124, -15.5464,  15.0440],
        [-17.7185, -14.4813, -14.2603,  14.4314, -12.9631,  13.6840, -15.1793,
          14.9599, -13.6359,  16.6044],
        [-15.6717, -15.4481, -16.7212,  13.0203, -15.8214,  15.9728, -14.5531,
          17.0585, -16.1497,  15.4623],
        [-15.6596, -16.0321, -16.3203,  16.5812, -15.0806,  14.1557, -15.8440,
          15.3820, -16.2132,  14.3963],
        [ 14